In [2]:
from langchain.vectorstores.cassandra import Cassandra
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.llms import  OpenAI
from langchain.embeddings import OpenAIEmbeddings

# Support for Datasets Retrival from HF
from datasets import  load_dataset

# With CassIO, the Engine powering the Astra DB integration in LangChain,
# Also initialize the DB connection:
import cassio

In [4]:
from PyPDF2 import  PdfReader
from dotenv import load_dotenv
import os
load_dotenv()

True

In [5]:
ASTRA_DB_ID = os.getenv('ASTRA_DB_ID')
ASTRA_DB_APP_TOKEN = os.getenv('ASTRA_DB_APP_TOKEN')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

In [7]:
# Load the PDF File
pdfreader = PdfReader('bis_fr.pdf')

In [8]:
from typing_extensions import Concatenate
# Read Text from the PDF
raw_text = ''
for i,page in enumerate(pdfreader.pages):
    content = page.extract_text()
    if content:
        raw_text += content

In [13]:
# Init Connection to the DB
cassio.init(token=ASTRA_DB_APP_TOKEN,database_id=ASTRA_DB_ID)

In [14]:
# Create the LangChain embedding and LLM objects
llm = OpenAI(openai_api_key=OPENAI_API_KEY)
embedding = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

In [15]:
# Create your LangChain vector store
vector_store = Cassandra(
    embedding=embedding,
    table_name="pdf_qry_demo",
    session=None,
    keyspace=None
)

In [16]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=800,
    chunk_overlap=500,
    length_function=len
)
texts = text_splitter.split_text(raw_text)

In [17]:
# Load the dataset into the vector store
vector_store.add_texts(texts)
print("Inserted to the DB")
vector_store_index = VectorStoreIndexWrapper(vectorstore=vector_store)

Inserted to the DB


In [24]:
# Run
query_text = input("Ask a question").strip()

print(query_text)
answer = vector_store_index.query(query_text,llm=llm).strip()
print(answer)

Qu elle sont les seuils a ne pas dépasser comme charges bureau si ma societe genere 50.000 mad par mois
Selon l'article 35 de la loi fiscale marocaine, les charges déductibles ne doivent pas dépasser 50 000 dirhams par mois pour chaque fournisseur de services, dans la limite de 5 000 dirhams pour chaque opération. Cela s'applique notamment aux frais liés à la location de compteurs d'eau et d'électricité, les ventes et prestations de services des opérateurs de télécommunication, les services d'agents démarcheurs ou courtiers d'assurances, et les opérations de services dont le montant est inférieur ou égal à 5 000 dirhams. Au-delà de ces seuils, une amende de 15% du montant de la taxe sur la valeur ajoutée peut être appliquée.
